In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [11]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Input, Subtract, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf

import cv2
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [4]:
from keras.backend.tensorflow_backend import set_session
from utils import limited_gpu_memory_session
set_session(limited_gpu_memory_session(0.8))

In [5]:
DATA_DIR = '/home/Drive2/rishabh/'
INIT_WEIGHTS = os.path.join(DATA_DIR, 'init_weights_omniglot.hdf5')
CHECKPOINTED_WEIGHTS = os.path.join(DATA_DIR, 'checkpointed_weights_omniglot.hdf5')
INIT_WEIGHTS = os.path.join(DATA_DIR, 'init_weights_omniglot.hdf5')

In [6]:
from keras.preprocessing.image import ImageDataGenerator
image_width = 105
image_height = 105
image_size = (image_width, image_height)
GEN_BATCH_SIZE = 512

train_datagen = ImageDataGenerator(rescale=1.0/255,
                rotation_range=10, width_shift_range=0.2,
                height_shift_range = 0.2, shear_range=0.2,
                zoom_range=0.2)

train_dir = os.path.join(DATA_DIR, 'omniglot/python/images_background')

train_generator = train_datagen.flow_from_directory(
        train_dir,  target_size=image_size,
        batch_size=GEN_BATCH_SIZE,
        class_mode='sparse', color_mode="grayscale",
        shuffle=True)

test_dir = os.path.join(DATA_DIR, 'omniglot/python/images_evaluation')

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  target_size=image_size, # this is the target directory
        batch_size=GEN_BATCH_SIZE*4, color_mode="grayscale",
        class_mode='sparse')

Found 19280 images belonging to 30 classes.
Found 13180 images belonging to 20 classes.


# Build the convolutional neural network

In [22]:
from keras.regularizers import l2

W_init = 'glorot_uniform'
input_shape = (105, 105, 1)
reg, dropout = 0, 0.5
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64, (10,10), activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init, kernel_regularizer=l2(reg)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128, (7,7), activation='relu', kernel_regularizer=l2(reg),
                   kernel_initializer=W_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init, kernel_regularizer=l2(reg)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init, kernel_regularizer=l2(reg)))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="selu",kernel_regularizer=l2(reg),kernel_initializer=W_init))
convnet.add(Dropout(dropout))
convnet.add(Dense(2048,activation="selu",kernel_regularizer=l2(reg),kernel_initializer=W_init))
convnet.add(Dropout(dropout))
convnet.add(Dense(1024,activation="selu",kernel_regularizer=l2(reg),kernel_initializer=W_init))
# convnet.add(Lambda(lambda x : tf.nn.l2_normalize(x, dim=0, epsilon=1e-20)))
print(convnet.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 9, 9, 128)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 6, 6, 256)         524544    
__________

## Define the loss functions for the siamese and triplet network

In [23]:
def l2_norm(x):
    return K.sqrt(K.sum(K.square(x)))

MARGIN = 0.2
def triplet_loss(y_true, y_pred): # 
    return K.mean(K.maximum(0.0, y_pred + MARGIN) - y_true * 0, axis = -1)

## Define the siamese network built using the conv net defined above

In [24]:
from keras import layers
# encode each of the two inputs into a vector with the convnet
left_input, right_input = Input(input_shape), Input(input_shape)
encoded_l, encoded_r  = convnet(left_input), convnet(right_input)
# merge two encoded inputs with a distance metric
diff = Subtract()([encoded_l,encoded_r])
prediction = Lambda(l2_norm, output_shape=(1,))(diff)
# prediction = layers.Dot(axes = -1, normalize=False)([encoded_l, encoded_r])
# prediction = Activation('sigmoid')(prediction)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
siamese_net.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 105, 105, 1)   0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 105, 105, 1)   0                                            
____________________________________________________________________________________________________
sequential_9 (Sequential)        (None, 1024)          49436480    input_5[0][0]                    
                                                                   input_6[0][0]                    
____________________________________________________________________________________________________
subtract_1 (Subtract)            (None, 1024)          0           sequential_9[1][0]      

## Define the triplet network

In [25]:
input_triples = [Input(input_shape) for _ in range(3)]
pos_output = siamese_net([input_triples[0], input_triples[1]])
neg_output = siamese_net([input_triples[0], input_triples[2]])
diff = Subtract()([neg_output, pos_output])
triplet_net = Model(inputs = input_triples, outputs = diff)
triplet_net.summary()
triplet_net.save_weights(INIT_WEIGHTS)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 105, 105, 1)   0                                            
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 105, 105, 1)   0                                            
____________________________________________________________________________________________________
input_8 (InputLayer)             (None, 105, 105, 1)   0                                            
____________________________________________________________________________________________________
model_2 (Model)                  (None, 1)             49436480    input_7[0][0]                    
                                                                   input_8[0][0]           

### Create the data generator to load batches of data

In [26]:
import utils; reload(utils)
from utils import TripletGenerator

NUM_TRAIN_TRIPLETS = 4096
NUM_VAL_TRIPLETS = 10000
BATCH_SIZE = 128
datagen = TripletGenerator(train_generator, test_generator, NUM_VAL_TRIPLETS, 
                           batch_sz=BATCH_SIZE, num_train_triplets=NUM_TRAIN_TRIPLETS)

In [27]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
              patience=3, verbose = 1, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss',
                              min_delta=1e-4,
                              patience=25,
                              verbose=0, mode='auto')
checkpointer = ModelCheckpoint(filepath=CHECKPOINTED_WEIGHTS, verbose=1, save_best_only=True, monitor='val_loss')

In [28]:
from keras.optimizers import Adam
adam = Adam(2e-4)
triplet_net.compile(loss=triplet_loss, optimizer=adam)
triplet_net.load_weights(INIT_WEIGHTS)

In [29]:
TRAIN_SIZE = 19280
STEPS_PER_EPOCH = (TRAIN_SIZE//GEN_BATCH_SIZE) * (NUM_TRAIN_TRIPLETS // (BATCH_SIZE))
VALIDATION_STEPS = NUM_VAL_TRIPLETS//BATCH_SIZE

In [ ]:
# triplet_net.load_weights(CHECKPOINTED_WEIGHTS)
history = triplet_net.fit_generator(
        datagen.next_train(),
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=500,
        validation_data=datagen.next_val(),
        validation_steps=VALIDATION_STEPS,
        callbacks = [reduce_lr, checkpointer, early_stopping])

Epoch 1/500
1184/1184 [==============================] - 254s - loss: 0.2710 - val_loss: 0.2000
Epoch 2/500
1184/1184 [==============================] - 250s - loss: 0.2482 - val_loss: 0.2000
Epoch 3/500
1184/1184 [==============================] - 250s - loss: 0.2975 - val_loss: 0.2000
Epoch 4/500
1184/1184 [==============================] - 250s - loss: 0.3224 - val_loss: 0.2130
Epoch 5/500
 496/1184 [===========>..................] - ETA: 143s - loss: 0.4027

In [24]:
triplet_net.load_weights(CHECKPOINTED_WEIGHTS)

In [25]:
siamese_model_path = os.path.join(DATA_DIR, 'siamese_omniglot')
siamese_net.save(siamese_model_path)